# How to use this for just plotting
In case you haven't worked with the repo but want to get these plots, there are some preliminary steps you'll need to follow.
1. For now, clone the whole repo (`git clone https://github.com/pepborrell/cool-chic.git`). It takes a while and it's a little heavy but it will make things so much easier.
2. Create the environment to run all code in the repo with `uv run python`. If you don't have uv, install it with `curl -LsSf https://astral.sh/uv/install.sh | sh` (or `pip install uv`). Use that environment to run this notebook.
3. Theoretically, all necessary data is already in the repo.

You can now plot all you need, it should work.

# Plot 1: RD plots for avgs in a dataset and some images

In [ ]:
from collections import defaultdict
from typing import Literal

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib.patches import Patch

from coolchic.eval.bd_rate import bd_rates_summary_anchor_name
from coolchic.eval.hypernet import (
    get_hypernet_flops,
    plot_hypernet_rd,
    plot_hypernet_rd_avg,
)
from coolchic.eval.results import SummaryEncodingMetrics, parse_hypernet_metrics
from coolchic.hypernet.hypernet import DeltaWholeNet, NOWholeNet
from coolchic.utils.paths import COOLCHIC_REPO_ROOT, DATA_DIR, RESULTS_DIR

sns.set_theme(context="notebook", style="whitegrid")
plt.rcParams["figure.dpi"] = 300

In [ ]:
import sys

print(sys.executable)

In [ ]:
sweep_path = RESULTS_DIR / "exps/delta-hn/longer-ups-best-orange/"
compare_no_path = RESULTS_DIR / "exps/no-cchic/orange-best/"

In [ ]:
def rd_plots_from_dataset(dataset: Literal["kodak", "clic20-pro-valid"]) -> None:
    import pandas as pd

    metrics = parse_hypernet_metrics(sweep_path, dataset=dataset, premature=True)
    df = pd.concat(
        [
            pd.DataFrame(
                [s.model_dump() for seq_res in metrics.values() for s in seq_res]
            ).assign(anchor="hypernet (ours)"),
            pd.DataFrame(  # For comparison with no hypernet.
                [
                    s.model_dump()
                    for seq_res in parse_hypernet_metrics(
                        compare_no_path, dataset, premature=True
                    ).values()
                    for s in seq_res
                ]
            ).assign(anchor="n-o cool-chic"),
        ],
    ).sort_values(by=["seq_name", "lmbda"])  # So plots come out nice and in order.

    plot_hypernet_rd_avg(df, dataset=dataset)

    all_images = sorted(list((DATA_DIR / dataset).glob("*.png")), key=lambda x: x.stem)
    for img in all_images[:5]:
        seq_name = img.stem
        plot_hypernet_rd(seq_name, df, dataset=dataset)

In [ ]:
def rd_plots_from_dataset_print(dataset: Literal["kodak", "clic20-pro-valid"]) -> None:
    import pandas as pd

    metrics = parse_hypernet_metrics(sweep_path, dataset=dataset, premature=True)
    df = pd.concat(
        [
            pd.DataFrame(
                [s.model_dump() for seq_res in metrics.values() for s in seq_res]
            ).assign(anchor="hypernet (ours)"),
            pd.DataFrame(  # For comparison with no hypernet.
                [
                    s.model_dump()
                    for seq_res in parse_hypernet_metrics(
                        compare_no_path, dataset, premature=True
                    ).values()
                    for s in seq_res
                ]
            ).assign(anchor="n-o cool-chic"),
        ],
    ).sort_values(by=["seq_name", "lmbda"])  # So plots come out nice and in order.

    from pathlib import Path
    from typing import Any

    def split_row(row: str) -> list[Any]:
        split_char = " "
        if "\t" in row:
            split_char = "\t"
        return [elem.strip() for elem in row.split(split_char) if elem != ""]

    def parse_result_summary(
        summary_file: Path,
    ) -> dict[str, list[SummaryEncodingMetrics]]:
        with open(summary_file, "r") as f:
            metric_names = split_row(f.readline().strip())
            raw_metrics = f.readlines()
        results = defaultdict(list)
        for line in raw_metrics:
            line_metrics = {n: v for n, v in zip(metric_names, split_row(line))}
            results[line_metrics["seq_name"]].append(
                SummaryEncodingMetrics(**line_metrics)
            )
        return dict(results)

    def result_summary_to_df(summary_path: Path) -> pd.DataFrame:
        summaries = parse_result_summary(summary_path)
        all_data = []
        for seq_name in summaries:
            all_data.extend([s.model_dump() for s in summaries[seq_name]])
        return pd.DataFrame(all_data)

    def compare_dataset_res(results: pd.DataFrame, dataset) -> pd.DataFrame:
        from coolchic.utils.paths import ALL_ANCHORS

        res_sums = []
        for anchor, results_path in ALL_ANCHORS[dataset].items():
            df = result_summary_to_df(results_path)
            df["anchor"] = anchor
            # delete columns that are all nan
            df = df.dropna(axis=1, how="all")
            res_sums.append(df)

        assert "anchor" in results.columns, "Anchor column not found in results"
        res_sums.append(results)
        all_df = pd.concat(res_sums)
        return all_df

    def plot_hypernet_rd_avg(results: pd.DataFrame, dataset):
        """Plots the average RD plot for the whole dataset in results."""
        all_df = compare_dataset_res(results, dataset)
        mean_df = (
            all_df.groupby(["anchor", "lmbda"])
            .agg({"rate_bpp": "mean", "psnr_db": "mean"})
            .reset_index()
        ).sort_values(by=["anchor", "rate_bpp"])
        with pd.option_context("display.max_rows", None, "display.max_columns", None):
            print(mean_df)
        # Assuming mean_df is your DataFrame with columns: anchor, rate_bpp, psnr_db
        anchors = mean_df["anchor"].unique()

        for anchor in anchors:
            df_anchor = mean_df[mean_df["anchor"] == anchor][["rate_bpp", "psnr_db"]]
            df_anchor.to_csv(
                f"{anchor.replace(' ', '_').replace('(', '').replace(')', '')}.csv",
                index=False,
            )

        fig, ax = plt.subplots()
        sns.lineplot(
            mean_df,
            x="rate_bpp",
            y="psnr_db",
            hue="anchor",
            marker="o",
            markeredgecolor="none",
            ax=ax,
            sort=False,
        )
        ax.set_title(
            f"Average RD curve. Dataset: {'CLIC2020' if dataset == 'clic20-pro-valid' else 'kodak'}"
        )
        ax.set_xlabel("Rate (bpp)")
        ax.set_ylabel("PSNR (dB)")
        return fig, ax

    fig, ax = plot_hypernet_rd_avg(df, dataset=dataset)
    plt.savefig("kodak.png")
    # all_images = sorted(list((DATA_DIR / dataset).glob("*.png")), key=lambda x: x.stem)
    # for img in all_images[:5]:
    #     seq_name = img.stem
    #     plot_hypernet_rd(seq_name, df, dataset=dataset)

In [ ]:
# df = rd_plots_from_dataset("kodak")  # get data and run existing plot code if needed
df = rd_plots_from_dataset_print("clic20-pro-valid")

In [ ]:
rd_plots_from_dataset("clic20-pro-valid")

# Plot 2: flops vs BD-rate

In [ ]:
import matplotlib.pyplot as plt

# import pandas as pd
import seaborn as sns

# from coolchic.eval.hypernet import get_hypernet_flops
# from coolchic.hypernet.hypernet import DeltaWholeNet, NOWholeNet

sns.set_theme(context="notebook", style="whitegrid")

In [ ]:
flops = {
    "coolchic": (
        coolchic_fwd := get_hypernet_flops(DeltaWholeNet, get_coolchic_flops=True)
    ),
    "coolchic_step": 3 * coolchic_fwd,
    "N-O Cool-chic": get_hypernet_flops(NOWholeNet),
    "Hypernet (ours)": get_hypernet_flops(DeltaWholeNet),
}

bd_rates = {
    "kodak": {
        "N-O Cool-chic": 0,
        "Hypernet (ours)": 0,
    },
    "clic20-pro-valid": {
        "N-O Cool-chic": 0,
        "Hypernet (ours)": 0,
    },
}

metrics_dfg = pd.DataFrame(
    [
        (dataset, method, bd_rate, flops[method])
        for dataset, bd_rates in bd_rates.items()
        for method, bd_rate in bd_rates.items()
    ],
    columns=["dataset", "method", "bd_rate", "flops"],
)
metrics_dfg["num_coolchic_steps"] = metrics_dfg["flops"] / flops["coolchic_step"]
metrics_dfg.head()

In [ ]:
fig, ax = plt.subplots()
sns.scatterplot(
    data=metrics_dfg[metrics_dfg["dataset"] == "clic20-pro-valid"],
    x="num_coolchic_steps",
    y="bd_rate",
    hue="method",
    ax=ax,
)
ax.set_title("BD Rate vs. Equivalent number of Cool-chic Steps, CLIC20 dataset")
sns.despine()

# Plot 4: show bd rate vs flops as training progresses

In [ ]:
flops = {
    "coolchic": (
        coolchic_fwd := get_hypernet_flops(DeltaWholeNet, get_coolchic_flops=True)
    ),
    "coolchic_step": 3 * coolchic_fwd,
    "N-O Cool-chic": get_hypernet_flops(NOWholeNet),
    "Hypernet (ours)": get_hypernet_flops(DeltaWholeNet),
}

bd_rates = {
    "kodak": {
        "N-O Cool-chic": 24.4463703313217,
        "Hypernet (ours)": 23.089282405055148,
    },
    "clic20-pro-valid": {
        "N-O Cool-chic": 19.338375256558283,
        "Hypernet (ours)": 14.948009862621655,
    },
}


metrics_df = pd.DataFrame(
    [
        (dataset, method, bd_rate, flops[method])
        for dataset, bd_rates in bd_rates.items()
        for method, bd_rate in bd_rates.items()
    ],
    columns=["dataset", "method", "bd_rate", "flops"],
)
metrics_df["num_coolchic_steps"] = metrics_df["flops"] / flops["coolchic_step"]
metrics_df.head()

In [ ]:
dataset = "clic20-pro-valid"
# dataset = "kodak"
CONFIG_NUM_TO_LMBDA = {
    "00": 0.0001,
    "01": 0.0002,
    "02": 0.0004,
    "03": 0.001,
    "04": 0.004,
    "05": 0.02,
}
# We filter out the methods that are not interesting for our analysis.
interesting_methods = ["orange-best", "longer-ups-best-orange"]

finetuning_dir = RESULTS_DIR / "finetuning" / dataset
finetuning_df = pd.concat(
    [
        pd.read_csv(file)
        for file in finetuning_dir.glob("**/*.csv")
        if file.parent.stem in interesting_methods
    ]
)
finetuning_df = finetuning_df.reset_index(drop=True)
finetuning_df["anchor"] = finetuning_df["anchor"].apply(
    lambda x: "N-O Cool-chic finetuning" if "NO" in x else "Hypernet finetuning"
)

In [ ]:
def get_bd_rate_from_df(df: pd.DataFrame) -> float:
    metrics = defaultdict(list)
    for _, row in df.iterrows():
        assert "seq_name" in row, "seq_name column is missing in the DataFrame"
        metrics[row["seq_name"]].append(
            SummaryEncodingMetrics(
                seq_name=row["seq_name"],
                lmbda=row["lmbda"],
                rate_bpp=row["rate_bpp"],
                psnr_db=row["psnr_db"],
            )
        )

    bds = bd_rates_summary_anchor_name(
        metrics, anchor="hm", dataset=dataset, only_latent_rate=False
    )
    assert len(bds) == 1, "Expected exactly one BD rate result."
    return list(bds.values())[0]


bd_df = (
    finetuning_df.groupby(["n_itr", "seq_name", "anchor"], group_keys=False)
    .apply(get_bd_rate_from_df)
    .reset_index()
    .rename(columns={0: "bd_rate"})
)
bd_df["num_coolchic_steps"] = (
    bd_df["n_itr"] + flops["N-O Cool-chic"] / flops["coolchic_step"]
)

In [ ]:
fig, ax = plt.subplots()
sns.scatterplot(
    data=metrics_df[metrics_df["dataset"] == dataset],
    x="num_coolchic_steps",
    y="bd_rate",
    hue="method",
    ax=ax,
)
avg_bd_df = (
    bd_df.groupby(["anchor", "n_itr"])
    .agg({"bd_rate": "mean", "num_coolchic_steps": "mean"})
    .reset_index()
)
# Add the BD rates from the init points to give an impression of finetuning from the start.
avg_bd_df = pd.concat(
    [
        avg_bd_df,
        metrics_df[metrics_df["dataset"] == dataset][
            ["num_coolchic_steps", "bd_rate", "method"]
        ]
        .rename(columns={"method": "anchor"})
        .assign(
            anchor=lambda df: df["anchor"]
            .apply(lambda x: x.strip(" (ours)"))
            .apply(lambda x: f"{x} finetuning")
        )
        .assign(n_itr=0),
    ]
)
# Filter out the points in finetuning that are higher than the init points.
avg_bd_df = avg_bd_df[avg_bd_df.n_itr != 100]

# Need this to match the hue order with the scatter plot.
methods_used = [
    f"{name.strip(' (ours)')} finetuning" for name in metrics_df.method.unique()
]
sns.lineplot(
    data=avg_bd_df,
    x="num_coolchic_steps",
    y="bd_rate",
    hue="anchor",
    hue_order=methods_used,
    ax=ax,
)
ax.set_title(
    f"BD Rate vs. Equivalent number of Cool-chic Steps, averaged over {'CLIC2020' if dataset == 'clic20-pro-valid' else 'Kodak' if dataset == 'kodak' else None} dataset"
)
ax.set_xlabel("Equivalent number of Cool-chic steps")
ax.set_ylabel("average BD Rate (vs. HEVC)")
sns.despine()

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

fig, ax = plt.subplots()

# Scatter plot of raw data
scatter_data = metrics_df[metrics_df["dataset"] == dataset]
sns.scatterplot(
    data=scatter_data,
    x="num_coolchic_steps",
    y="bd_rate",
    hue="method",
    ax=ax,
)

# Print scatter plot data
print("=== Scatter Data ===")
print(scatter_data[["num_coolchic_steps", "bd_rate", "method"]])

# Create averaged lineplot data
avg_bd_df = (
    bd_df.groupby(["anchor", "n_itr"])
    .agg({"bd_rate": "mean", "num_coolchic_steps": "mean"})
    .reset_index()
)

# Add initial points (n_itr = 0) to the average data for visualizing finetuning starting points
init_points = (
    metrics_df[metrics_df["dataset"] == dataset][
        ["num_coolchic_steps", "bd_rate", "method"]
    ]
    .rename(columns={"method": "anchor"})
    .assign(
        anchor=lambda df: df["anchor"]
        .apply(lambda x: x.strip(" (ours)"))
        .apply(lambda x: f"{x} finetuning"),
        n_itr=0,
    )
)

# Concatenate init points to averaged data
avg_bd_df = pd.concat([avg_bd_df, init_points])

# Filter out final iteration points if needed
avg_bd_df = avg_bd_df[avg_bd_df.n_itr != 100]

# Define hue order to match scatter plot
methods_used = [
    f"{name.strip(' (ours)')} finetuning" for name in metrics_df.method.unique()
]

# Plot average BD rate over steps
sns.lineplot(
    data=avg_bd_df,
    x="num_coolchic_steps",
    y="bd_rate",
    hue="anchor",
    hue_order=methods_used,
    ax=ax,
)

# Print line plot data
print("=== Line Data ===")
line_df = avg_bd_df.assign(mac_pixel=avg_bd_df["num_coolchic_steps"] * 4290)[
    ["num_coolchic_steps", "mac_pixel", "bd_rate", "n_itr", "anchor"]
]
print(line_df.columns.tolist())
for row in line_df.iterrows():
    # Print each row as a space-separated string
    print(" ".join(map(str, row[1].values)))

# Titles and labels
ax.set_title(
    f"BD Rate vs. Equivalent number of Cool-chic Steps, averaged over {'CLIC2020' if dataset == 'clic20-pro-valid' else 'Kodak' if dataset == 'kodak' else None} dataset"
)
ax.set_xlabel("Equivalent number of Cool-chic steps")
ax.set_ylabel("average BD Rate (vs. HEVC)")

# Style
sns.despine()

In [ ]:
# MACs per pixel of a Cool-chic training step.
flops["coolchic_step"] / (512**2)

In [ ]:
# MACs per pixel of a Cool-chic forward pass.
{name: flops[name] / (512**2) for name in flops}

# Plot 5: extra rate and benefits of adding each part

In [ ]:
dataset = "clic20-pro-valid"
ablation_dir = COOLCHIC_REPO_ROOT / "switch-ablation-exps"

all_ablation_metrics = {}
for dir in ablation_dir.iterdir():
    parts = dir.stem.split("_")
    dir_metrics = pd.DataFrame(
        [
            s.model_dump()
            for seq_res in parse_hypernet_metrics(dir, dataset=dataset).values()
            for s in seq_res
        ]
    ).assign(variant=dir.name if dir.name != "none" else "baseline")
    all_ablation_metrics[dir.name] = dir_metrics

all_metrics = pd.concat(
    all_ablation_metrics.values(),
    ignore_index=True,
).sort_values(by=["seq_name", "lmbda"])

In [ ]:
agg_df = all_metrics.groupby(["variant", "lmbda"]).agg(
    {
        "rate_nn_bpp": "mean",
        "psnr_db": "mean",
        "rate_latent_bpp": "mean",
    }
)

# First group the baseline rows by 'lmbda' and subtract them from each group.
baseline = agg_df.loc["baseline"]


def subtract_baseline(group):
    lmbda_val = group.name
    base_row = baseline.loc[lmbda_val]
    return group - base_row


# Apply the subtraction grouped by 'lmbda'
diffs = agg_df.groupby("lmbda").apply(subtract_baseline, include_groups=False)
diffs = (
    diffs.reset_index(level=0, drop=True)
    .drop("baseline", level=0)
    .reset_index()
    .sort_values(
        by=["variant", "lmbda"],
        key=lambda x: x.map(
            {
                "with ARM modulations": 0,
                "with synthesis modulations": 1,
                "full hypernetwork (ours)": 2,
            }
        ),
    )
    .rename(
        columns={
            "rate_nn_bpp": "rate_nn_bpp_diff",
            "psnr_db": "psnr_db_diff",
            "rate_latent_bpp": "rate_latent_bpp_diff",
        }
    )
)
diffs

In [ ]:
# Melt the DataFrame for seaborn
plot_data = diffs.melt(
    id_vars=["variant"],
    value_vars=["rate_nn_bpp_diff", "rate_latent_bpp_diff", "psnr_db_diff"],
    var_name="metric",
    value_name="value",
)

# Separate data for the two axes
bpp_data = plot_data[plot_data["metric"].str.contains("bpp")]
psnr_data = plot_data[plot_data["metric"].str.contains("psnr")]

# Pivot the data and remove the baseline variant (its diff is always 0)
diffs_pivot = diffs.set_index("variant").loc[
    ["with ARM modulations", "with synthesis modulations", "full hypernetwork (ours)"]
]

In [ ]:
def plot_ablation(diff_pivot_df: pd.DataFrame, lmbda_val: float | None = None) -> None:
    print("\n--- LaTeX Table ---")
    print(
        diff_pivot_df.to_latex(
            float_format="%.4f", column_format="lccc", bold_rows=True
        )
    )

    # --- 4. Create the Plot ---
    fig, ax1 = plt.subplots(figsize=(6, 3))
    ax2 = ax1.twinx()

    # --- Bar properties ---
    variants = diff_pivot_df.index
    x_pos = np.arange(len(variants))  # The label locations [0, 1, 2]
    bar_width = 0.10  # Set the desired width of the bars
    offsets = np.array([-bar_width, 0, bar_width])  # Offsets for the three bars

    # Define colors
    # Dutch field and blue to yellow color palettes
    # https://www.heavy.ai/blog/12-color-palettes-for-telling-better-stories-with-your-data
    bpp1_color = "#115f9a"
    bpp2_color = "#22a7f0"
    psnr_color = "#e60049"  # A slightly different red

    # --- Plotting the bars manually ---
    # Plot BPP stems on ax1
    markerline, stemlines, baseline = ax1.stem(
        x_pos + offsets[0], diff_pivot_df["rate_nn_bpp_diff"]
    )
    plt.setp(markerline, marker="o", markersize=8, color=bpp1_color)
    plt.setp(stemlines, color=bpp1_color)
    plt.setp(baseline, visible=False)

    markerline, stemlines, baseline = ax1.stem(
        x_pos + offsets[1], diff_pivot_df["rate_latent_bpp_diff"]
    )
    plt.setp(markerline, marker="o", markersize=8, color=bpp2_color)
    plt.setp(stemlines, color=bpp2_color)
    plt.setp(baseline, visible=False)

    # Plot PSNR stems on ax2
    markerline, stemlines, baseline = ax2.stem(
        x_pos + offsets[2], diff_pivot_df["psnr_db_diff"]
    )
    plt.setp(markerline, marker="o", markersize=8, color=psnr_color)
    plt.setp(stemlines, color=psnr_color)
    plt.setp(baseline, visible=False)

    # --- 5. Customize the Plot ---
    ax1.set_title(
        f"Ablation Study: Metric changes with respect to N-O Cool-chic. Lambda: {lmbda_val if lmbda_val is not None else 'all'}",
        fontsize=12,
        pad=10,
    )
    ax1.set_xlabel("Model Variant", fontsize=10)
    ax1.set_ylabel(
        "$\\Delta$ bpp ($\\leftarrow$ better)", fontsize=10, color=bpp1_color
    )
    ax2.set_ylabel(
        "$\\Delta$ PSNR (dB) ($\\rightarrow$ better)", fontsize=10, color=psnr_color
    )
    ax1.set_xticks(x_pos)
    ax1.set_xticklabels(variants, fontsize=8)
    ax1.tick_params(axis="y", labelcolor=bpp1_color)
    ax2.tick_params(axis="y", labelcolor=psnr_color)
    ax2.spines["left"].set_color(bpp1_color)
    ax2.spines["right"].set_color(psnr_color)

    # --- 6. Align the zero-lines of the two axes (The Fix) ---
    # Get the current limits
    y1_min, y1_max = ax1.get_ylim()
    y2_min, y2_max = ax2.get_ylim()

    # Calculate the largest absolute value for each axis
    abs_max1 = max(abs(y1_min), abs(y1_max))
    abs_max2 = max(abs(y2_min), abs(y2_max))

    # Set symmetrical limits on both axes with a 10% margin
    ax1.set_ylim(-abs_max1 * 1.1, abs_max1 * 1.1)
    ax2.set_ylim(-abs_max2 * 1.1, abs_max2 * 1.1)

    # Now that they are aligned, one zero-line is sufficient
    ax1.axhline(0, color="black", linewidth=1.0, linestyle="--", zorder=0)

    # --- 6. Create a Manual Legend ---
    legend_elements = [
        Patch(
            facecolor=bpp1_color,
            edgecolor="black",
            label="$\\Delta$ Modulation rate (bpp, low is better)",
        ),
        Patch(
            facecolor=bpp2_color,
            edgecolor="black",
            label="$\\Delta$ Latent rate (bpp, low is better)",
        ),
        Patch(
            facecolor=psnr_color,
            edgecolor="black",
            label="$\\Delta$ PSNR (dB, high is better)",
        ),
    ]
    ax1.legend(
        handles=legend_elements,
        loc="lower center",
        # bbox_to_anchor=(0.5, 0.15),
        # ncol=3,
        fancybox=True,
        shadow=False,
        fontsize=7,
    )

    # Adjust axis limits to prevent bars from touching the edges
    ax1_min, ax1_max = ax1.get_ylim()
    ax1.set_ylim(ax1_min * 1.15, ax1_max * 1.15)
    ax2_min, ax2_max = ax2.get_ylim()
    ax2.set_ylim(ax2_min * 1.15, ax2_max * 1.15)

    # plt.tight_layout(rect=[0, 0, 1, 0.95])  # Adjust layout to make room for the title
    plt.tight_layout()  # Adjust layout to make room for the title
    plt.show()

In [ ]:
for lmbd in diffs_pivot.lmbda.unique():
    plot_ablation(diffs_pivot[diffs_pivot.lmbda == lmbd], lmbda_val=lmbd)

In [ ]:
# same but in a table
for lmbd in diffs_pivot.lmbda.unique():
    print(f"Lambda: {lmbd}")
    print(
        diffs_pivot.loc[diffs_pivot.lmbda == lmbd][
            ["rate_nn_bpp_diff", "rate_latent_bpp_diff", "psnr_db_diff"]
        ].reset_index()
    )
    print("\n")

# How many images use modulations?

In [ ]:
def show_no_delta_perc(dataset):
    results_path = RESULTS_DIR / "exps/delta-hn/longer-ups-best-orange/"
    all_results = list(results_path.glob(f"**/{dataset}_results.csv"))

    df = pd.concat(
        [
            pd.read_csv(res_file).assign(config=res_file.parents[1].name)
            for res_file in all_results
        ],
        ignore_index=True,
    )

    # The ones that don't use a delta are shown as no_delta_noupsampling
    no_deltas = df.option_selected == "no_delta_noupsampling"
    return (
        df[no_deltas]
        .config.value_counts()
        .reset_index()
        .assign(percentage_ignored=lambda xdf: xdf["count"] / len(df.seq_name.unique()))
        .assign(perc_used=lambda df: 1 - df["percentage_ignored"])
        .assign(
            lmbda=lambda df: df["config"].apply(
                lambda x: CONFIG_NUM_TO_LMBDA[x.split("_")[-1]]
            )
        )
    )